In [7]:
import pandas as pd

id2label = pd.read_pickle('data/id2label_final.pkl')
label2id = pd.read_pickle('data/label2id_final.pkl')
submission_file = pd.read_csv('data/submission_file.csv')
test_ids_final = pd.read_csv('data/test_ids_final.csv')
test_ocr_clean = pd.read_pickle('data/test_ocr_clean.pkl')
train_ids_final = pd.read_csv('data/train_ids_final.csv')
train_labels_final = pd.read_pickle('data/train_labels_final.pkl')
train_set_ocr = pd.read_pickle('data/train_set_ocr.pkl')

In [21]:
text = test_ocr_clean['test_set/e9fdc30e-99bd-4df8-ac68-adab008dfc88.jpg']
text

'PITaxpl POLALSHPein 9220067289 8383511524 18 1 0 ofrin PIT-37 ZEZNANIE O WYSOKOSCI OSIĄGNIĘTEGO DOCHODU(P"NI9SIONEJ STRATY) roku podatkowym FoamuapnconvFoslol Oualn Alotry Goldn ua pchod[6oOcepuoronvr eryn Rrectypor Ipodlcelacecoocaomen Doauon npofconcha Pabnhoa NUb mych poonoton cgoiuchzsndach Aasosoqanual Prokadru pontonncc dadainota QOCPOda CIC Camch nuren] [lormaorOmiooatych Oodiodon lpnryhouoa) ouzoaMon soecanych proaunGIooc opodalrOuar L Gnnn to OooauriOolcrac UanhoocLodondocnoconmrcornchdaeg conuon Dochodow @sbaly (1ubeghrch Podslana Duwna At 5usL Ustawyzonn Z64pca 1991/ 0 podanu oochooonmodo  uycmnychiDz U 1 2021 r Doz 1128 Nanci dalet slawa sMadanta Od dnia 15 lulauo do dnla 30 khetniz naslepujnceoo po roku oodalkomm Zasuzezener Usiava art 45ust Lejsce sktidana Urzad korym mowa Usiawy Zwany dale] Uedem Wybor *0o,obuoco 0a . aoaoo disrosob  {oood Dradnany pmdrieny 2u1 Ustat = odru A MIEJSCE SKŁADANIA ZEZNANIA Uuad Skarbovn wNowm Sacz 0 Cclzlozonta lormularza Prhoe Rlors Mobe w

In [90]:
train = {'file1':('i want it That Way?',1),
             'file2':('TUHe may you do it!? /n come o 123-456-7890 n 93 94 2',2),
             'file3':('maybe$ you will Do IT 32 8 (123) 456-7890',1),
             'file4':('TUHe may you do it!? /n come on 93 94 2',2),
             'file5':('TUHe may you domm 123.456.7890 it!? /noo come omon 2',2),
             'file6':('TUHe may you kokodo i want it it!? /n co 123 456 7890',2),
             'file7':('okd mor uoowa o dzielo imfriofr r rwivnwroc v v',0),


      }

In [94]:
data_test = [{'file_name': key, 'string': value} for key, value in test_ocr_clean.items()]
df_test = pd.DataFrame(data_test)

In [97]:
text_vector

0       PITaxpl POLALSHPein 9220067289 8383511524 18 1...
1       Kooaek DalkoNana LiteRAV CZARNYM LLB NeBiesAV ...
2       Wypelnv Joonn Beyffenl Vazar FLENY Plsfl Delko...
3       PlTaxpl POLA JASNE WYPELN4 PoDATnIK POLA CieMN...
4       PlTaxpl POL A JASNE WypelNIA PodaTNix CienncKy...
                              ...                        
3495    Nancy Coleman 6831 Ridge Blvd. 414 BrockIyn, N...
3496    20733788/7 DEC_15.19393 40.22pM1 No.689 P.1/ T...
3497    AXXALS OF THE *EIv YORR ACADEMY OF sciecs VOLU...
3498    SENT BY :Mezz ina/Br 0wn   Inc_ 3-92 4:51PM 21...
3499                                  mal 4a Jeutl Uu + 1
Name: string, Length: 3500, dtype: object

In [109]:
pattern = r"\battachments\b"
target_phrase = 'attachments'

text_vector = df_test['string']

s = text_vector.apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
f = text_vector.apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)


print('basic:', s.sum())
print('fuzz:', f.sum())

basic: 20
fuzz: 21


In [91]:
data = [{'file_name': key, 'string': value[0], 'id': value[1]} for key, value in train.items()]
df = pd.DataFrame(data)
df

,file_name,string,id
0,file1,i want it That Way?,1
1,file2,TUHe may you do it!? /n come o 123-456-7890 n ...,2
2,file3,maybe$ you will Do IT 32 8 (123) 456-7890,1
3,file4,TUHe may you do it!? /n come on 93 94 2,2
4,file5,TUHe may you domm 123.456.7890 it!? /noo come ...,2
5,file6,TUHe may you kokodo i want it it!? /n co 123 4...,2
6,file7,okd mor uoowa o dzielo imfriofr r rwivnwroc v v,0


In [92]:
import re
from fuzzywuzzy import fuzz

df['string_length'] = df['string'].apply(len)

# Add a column for the count of capital letters
df['capital_letters_count'] = df['string'].apply(lambda x: sum(1 for c in x if c.isupper()))

# Add a column for the count of numbers
df['numbers_count'] = df['string'].apply(lambda x: sum(1 for c in x if c.isdigit()))
# Add a column for the count of question marks
df['question_marks_count'] = df['string'].apply(lambda x: x.count('?'))
# Add a column for the count of currency signs
df['currency_signs_count'] = df['string'].apply(lambda x: x.count('$') + x.count('€') + x.count('£'))

# american phone regex
pattern = r"\b\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"

# american address regex
pattern1 = r"^\d+\s+([a-zA-Z]+\s+)+[a-zA-Z]+,\s*[A-Za-z]{2}\s*\d{5}(?:-\d{4})?$"

# Add a column for the flag indicating pattern match
df['flag_phone'] = df['string'].apply(lambda x: 1 if re.search(pattern, x) else 0)

# Define the pattern to search for
pattern = r"\bi want it\b"

# Add a column for the flag indicating pattern match
df['flag_text'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)

target_phrase = "umowa o dzieło"

# Add a column for the flag indicating approximate match
df['flag_sentance_with_ratio'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)


df

,file_name,string,id,string_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,flag_phone,flag_text,flag_sentance_with_ratio
0,file1,i want it That Way?,1,19,2,0,1,0,0,1,0
1,file2,TUHe may you do it!? /n come o 123-456-7890 n ...,2,53,3,15,1,0,1,0,0
2,file3,maybe$ you will Do IT 32 8 (123) 456-7890,1,41,3,13,0,1,1,0,0
3,file4,TUHe may you do it!? /n come on 93 94 2,2,39,3,5,1,0,0,0,0
4,file5,TUHe may you domm 123.456.7890 it!? /noo come ...,2,52,3,11,1,0,1,0,0
5,file6,TUHe may you kokodo i want it it!? /n co 123 4...,2,53,3,10,1,0,1,1,0
6,file7,okd mor uoowa o dzielo imfriofr r rwivnwroc v v,0,47,0,0,0,0,0,0,1
